# Video style transfer

In [1]:
import cv2
import os
import pathlib

In [15]:
video_name = 'Titanic'

In [16]:
video_path = '{}/videos/'.format(os.getcwd())
video_pathlib = pathlib.Path(video_path)
if not video_pathlib.exists():
    pathlib.Path(video_pathlib).mkdir(parents=True, exist_ok=True)

In [17]:
frame_path = '{}/frames/{}/'.format(os.getcwd(), video_name)
frame_pathlib = pathlib.Path(frame_path)
if not frame_pathlib.exists():
    pathlib.Path(frame_pathlib).mkdir(parents=True, exist_ok=True)

convert raw video to image sequence

In [6]:
cap= cv2.VideoCapture(video_path+'{}.mp4'.format(video_name))
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite(frame_path+'frame{}.jpg'.format(str(i).zfill(4)),frame)
    i+=1

cap.release()
cv2.destroyAllWindows()

prepare the model

In [6]:
!wget https://s3-us-west-2.amazonaws.com/wengaoye/vgg19_normalised.npz
!wget https://s3-us-west-2.amazonaws.com/wengaoye/arbitrary_style_model_style-weight-2e0.zip
!unzip arbitrary_style_model_style-weight-2e0.zip

--2019-06-05 12:54:55--  https://s3-us-west-2.amazonaws.com/wengaoye/vgg19_normalised.npz
Resolving web.ucsd.edu (web.ucsd.edu)... 132.239.1.231, 132.239.1.230
Connecting to web.ucsd.edu (web.ucsd.edu)|132.239.1.231|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 160200914 (153M) [application/x-www-form-urlencoded]
Saving to: ‘vgg19_normalised.npz.1’

vgg19_normalised.np 100%[===================>] 152.78M  71.0MB/s    in 2.2s    

2019-06-05 12:54:58 (71.0 MB/s) - ‘vgg19_normalised.npz.1’ saved [160200914/160200914]

--2019-06-05 12:54:58--  https://s3-us-west-2.amazonaws.com/wengaoye/arbitrary_style_model_style-weight-2e0.zip
Resolving web.ucsd.edu (web.ucsd.edu)... 132.239.1.230, 132.239.1.231
Connecting to web.ucsd.edu (web.ucsd.edu)|132.239.1.230|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 59094122 (56M) [application/zip]
Saving to: ‘arbitrary_style_model_style-weight-2e0.zip.2’

arbitrary_style_mod 100%[==================

In [18]:
from __future__ import print_function

from infer import stylize
from utils import list_images

style transfer

In [25]:
style_path = '{}/style'.format(os.getcwd())
tf_frame_path = []
for img in os.listdir(style_path):
    tf_frame_path.append('{}/tf_frames/{}/{}/'.format(os.getcwd(), img[:-4], video_name))
    tf_frame_pathlib = pathlib.Path(tf_frame_path[-1])
    if not tf_frame_pathlib.exists():
        pathlib.Path(tf_frame_pathlib).mkdir(parents=True, exist_ok=True)

In [20]:
encoder_weights_path = 'vgg19_normalised.npz'
style_weights = [2.0]
model_save_paths = [
    'models/style_weight_2e0.ckpt',
]

In [29]:
# single style
content_imgs_path = list_images(frame_path)
style_imgs_path   = list_images(style_path)

for style_weight, model_save_path in zip(style_weights, model_save_paths):
    print('\n>>> Begin to stylize images with style weight: %.2f\n' % style_weight)

    for style, path in zip(style_imgs_path, tf_frame_path):
        stylize(content_imgs_path, style, path[:-1], 
                encoder_weights_path, model_save_path, 
                suffix='-' + str(style_weight))

print('\n>>> Successfully! Done all stylizing...\n')


>>> Begin to stylize images with style weight: 2.00

INFO:tensorflow:Restoring parameters from models/style_weight_2e0.ckpt

>>> Successfully! Done all stylizing...



In [26]:
mixed_path = '{}/tf_frames/mixed/'.format(os.getcwd())
mixed_pathlib = pathlib.Path(mixed_path)
if not mixed_pathlib.exists():
    pathlib.Path(mixed_pathlib).mkdir(parents=True, exist_ok=True)

convert to video

In [30]:
output_path = '{}/outputs'.format(os.getcwd())
output_pathlib = pathlib.Path(output_path)
if not output_pathlib.exists():
    pathlib.Path(output_pathlib).mkdir(parents=True, exist_ok=True)

In [40]:
# single style
for path, style in zip(tf_frame_path, [style[:-4] for style in os.listdir(style_path)]):
    tf_video_name = '{}/{}_{}.avi'.format(output_path, 'udnie', video_name)
    images = [img for img in os.listdir(path) if img.endswith(".jpg")]
    images = sorted(images, key=lambda img: int(img[5:9]))
    frame = cv2.imread(os.path.join(path, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(tf_video_name, 0, 16, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(path, image)))

    cv2.destroyAllWindows()
    video.release()